# به نام خدا

# فاز 1

In [18]:
import os
import cv2
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [20]:


# نسبت بزرگنمایی
SCALE_X = 1280 / 224
SCALE_Y = 1280 / 224

def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    plates = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        if name.lower() == 'vehicle plate':
            bndbox = obj.find('bndbox')
            xmin = int(int(bndbox.find('xmin').text) * SCALE_X)
            ymin = int(int(bndbox.find('ymin').text) * SCALE_Y)
            xmax = int(int(bndbox.find('xmax').text) * SCALE_X)
            ymax = int(int(bndbox.find('ymax').text) * SCALE_Y)
            plates.append((xmin, ymin, xmax, ymax))
    
    return plates

def extract_plates(image_path, annotation_path, output_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"خطا در خواندن تصویر: {image_path}")
        return
    
    plates = parse_annotation(annotation_path)
    if not plates:
        print(f"هیچ پلاکی در فایل آنوتیشن یافت نشد: {annotation_path}")
        return
    
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    for i, (xmin, ymin, xmax, ymax) in enumerate(plates):
        plate_img = image[ymin:ymax, xmin:xmax]
        
        output_file = os.path.join(output_path, f"{base_name}.png")
        cv2.imwrite(output_file, plate_img)

def process_dataset(base_dir, output_dir):
    images_dir = os.path.join(base_dir, "Vehicle Plates 1280x1280", "Vehicle Plates 1280x1280")
    annotations_dir = os.path.join(base_dir, "Vehicle Plates annotations", "Vehicle Plates annotations")
    
    if not os.path.exists(images_dir):
        print(f"پوشه تصاویر یافت نشد: {images_dir}")
        return
    if not os.path.exists(annotations_dir):
        print(f"پوشه آنوتیشن‌ها یافت نشد: {annotations_dir}")
        return
    
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    os.makedirs(output_dir, exist_ok=True)
    
    for img_file in tqdm(image_files, desc="پردازش تصاویر"):
        base_name = os.path.splitext(img_file)[0]
        image_path = os.path.join(images_dir, img_file)
        annotation_path = os.path.join(annotations_dir, f"{base_name}.xml")
        
        if os.path.exists(annotation_path):
            extract_plates(image_path, annotation_path, output_dir)
        else:
            print(f"فایل آنوتیشن یافت نشد: {annotation_path}")

if __name__ == "__main__":
    base_dir = "Plates2"
    output_dir = "output_phase1"
    
    process_dataset(base_dir, output_dir)
    
    print(f"✅ پردازش فاز اول با موفقیت به پایان رسید. پلاک‌ها در پوشه '{output_dir}' ذخیره شدند.")


پردازش تصاویر:   0%|          | 0/217 [00:00<?, ?it/s]

پردازش تصاویر: 100%|██████████| 217/217 [00:05<00:00, 36.84it/s]

✅ پردازش فاز اول با موفقیت به پایان رسید. پلاک‌ها در پوشه 'output_phase1' ذخیره شدند.


# فاز 2

### درست کردن کجی و سیاه سفید کردن

In [21]:
import os
import cv2
import numpy as np

In [22]:

input_folder = 'output_phase1'
output_folder = 'output_phase2'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def angle_from_min_area_rect(thresh):
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    largest = max(contours, key=cv2.contourArea)
    rect = cv2.minAreaRect(largest)
    angle = rect[-1]
    if angle < -45:
        angle += 90
    elif angle > 45:
        angle -= 90
    return angle

def angle_from_hough(gray):
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=30, maxLineGap=10)
    if lines is None:
        return None
    angles = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        if -45 < angle < 45:
            angles.append(angle)
    if not angles:
        return None
    return np.median(angles)

def correct_skew_combined(gray_image, angle_threshold=3):
    # Preprocess
    _, thresh = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    thresh = 255 - thresh

    # Step 1: try minAreaRect
    angle = angle_from_min_area_rect(thresh)

    # Step 2: fallback to HoughLines if angle too small or None
    if angle is None or abs(angle) < angle_threshold:
        angle = angle_from_hough(gray_image)

    # اگر همچنان زاویه معنادار نبود، تصویر را برگردان
    if angle is None or abs(angle) < angle_threshold:
        return gray_image

    # Rotate
    (h, w) = gray_image.shape
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(gray_image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# پردازش همه تصاویر
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        path = os.path.join(input_folder, filename)
        image = cv2.imread(path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        corrected = correct_skew_combined(gray)

        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, corrected)

print("✅ اصلاح زاویه با ترکیب minAreaRect + Hough انجام شد.")


✅ اصلاح زاویه با ترکیب minAreaRect + Hough انجام شد.


### کات دادن یک هشتم اول پلاک

In [23]:
import os
from PIL import Image

In [24]:

# مسیرهای ورودی و خروجی
input_dir = 'output_phase2'
output_dir = 'output_phase2.1'

# ساخت پوشه خروجی اگر وجود نداشت
os.makedirs(output_dir, exist_ok=True)

# پردازش هر تصویر در پوشه ورودی
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif')):
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path)

        # ابعاد تصویر
        width, height = img.size

        # عرض هر تکه
        slice_width = width // 8

        # برش تصویر: حذف تیکه اول (از x=0 تا x=slice_width)
        # نگه داشتن بخش باقی‌مانده (از x=slice_width تا x=width)
        cropped_img = img.crop((slice_width, 0, width, height))

        # ذخیره تصویر خروجی
        output_path = os.path.join(output_dir, filename)
        cropped_img.save(output_path)

print("پردازش تمام شد.")


پردازش تمام شد.


### باینری کردن عکس ها

In [41]:
import os
import cv2
import numpy as np

In [42]:


input_folder = 'output_phase2.1'
output_folder = 'output_phase2.2'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # 1. نویززدایی با فیلتر غیرمحلی (Non-local Means)
        denoised = cv2.fastNlMeansDenoising(img, h=10, templateWindowSize=7, searchWindowSize=21)

        # 2. افزایش کنتراست با CLAHE
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        contrast_enhanced = clahe.apply(denoised)

        # 3. نرم کردن با فیلتر گوسی
        blurred = cv2.GaussianBlur(contrast_enhanced, (3, 3), 0)

        # 4. آستانه‌گذاری تطبیقی برای باینری‌سازی
        binary_img = cv2.adaptiveThreshold(
            blurred,
            255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,
            11,  # اندازه بلوک
            2    # مقدار ثابت
        )

        # 5. حذف نویزهای کوچک با مورفولوژی
        kernel = np.ones((2, 2), np.uint8)
        processed = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernel)

        # 6. پر کردن حفره‌های داخل کاراکترها
        h, w = processed.shape
        flood_fill_mask = np.zeros((h + 2, w + 2), np.uint8)
        im_floodfill = processed.copy()

        # فیل کردن از یک نقطه‌ی مطمئن (گوشه‌ی تصویر)
        cv2.floodFill(im_floodfill, flood_fill_mask, (0, 0), 255)

        # یافتن حفره‌ها و پر کردن آن‌ها
        holes = cv2.bitwise_not(im_floodfill) & processed
        filled = processed | holes

        # 7. معکوس کردن: متن سفید، پس‌زمینه سیاه
        final_img = cv2.bitwise_not(filled)

        # 8. ذخیره تصویر خروجی
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, final_img)

print("پردازش پیشرفته و تبدیل به باینری با حفظ جزئیات انجام شد.")


پردازش پیشرفته و تبدیل به باینری با حفظ جزئیات انجام شد.


In [43]:


input_folder = 'output_phase2.1'
output_folder = 'output_phase2.2'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # 1. پیش‌پردازش اولیه با حفظ جزئیات
        denoised = cv2.fastNlMeansDenoising(img, h=5, templateWindowSize=7, searchWindowSize=21)
        
        # 2. افزایش کنتراست هوشمند
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(12,12))
        contrast_enhanced = clahe.apply(denoised)
        
        # 3. بهبود لبه‌ها با فیلتر ویژه
        blurred = cv2.bilateralFilter(contrast_enhanced, d=9, sigmaColor=75, sigmaSpace=75)
        
        # 4. روش ترکیبی آستانه‌گذاری
        # مرحله اول: Adaptive Threshold
        binary_adaptive = cv2.adaptiveThreshold(
            blurred,
            255,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY_INV,
            blockSize=9,
            C=2
        )
        
        # مرحله دوم: Otsu Threshold
        _, binary_otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # مرحله سوم: ترکیب با وزن‌دهی
        combined = cv2.addWeighted(binary_adaptive, 0.7, binary_otsu, 0.3, 0)
        
        # 5. بهبود نهایی با تکنیک Super-Resolution (اختیاری)
        # اگر opencv با contrib نصب باشد:
        # super_res = cv2.dnn_superres.DnnSuperResImpl_create()
        # super_res.readModel('EDSR_x4.pb')
        # super_res.setModel('edsr', 4)
        # final_img = super_res.upsample(combined)
        
        # 6. ذخیره نتیجه
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, cv2.bitwise_not(combined))

print("پردازش حرفه‌ای با حفظ حداکثر کیفیت انجام شد.")

پردازش حرفه‌ای با حفظ حداکثر کیفیت انجام شد.


### در اوردن کاراکتر های هر عکس

In [49]:
import os
import cv2
import numpy as np

In [ ]:

input_folder = "output_phase2.2"
output_folder = "output_phase2.3"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if not filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
        continue

    path = os.path.join(input_folder, filename)
    img_original = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img_original is None:
        print(f"⛔ نمی‌توان تصویر {filename} را خواند.")
        continue

    # بزرگنمایی ×3
    scale = 3
    img_highres = cv2.resize(img_original, (img_original.shape[1]*scale, img_original.shape[0]*scale), interpolation=cv2.INTER_CUBIC)

    # صاف‌سازی و باینری کردن
    img_blur = cv2.GaussianBlur(img_highres, (3, 3), 0)
    img_bin = cv2.adaptiveThreshold(
        img_blur, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV,
        15, 8
    )

    # حذف نویزهای کوچک
    kernel = np.ones((2, 2), np.uint8)
    img_bin = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernel)

    # پیدا کردن کانتور
    contours, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # فقط خارجی

    # فیلتر بر اساس ابعاد نسبی (هوشمندانه‌تر)
    boxes = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        area = cv2.contourArea(c)
        if w < 10 or h < 20:
            continue
        if w > img_highres.shape[1] * 0.5 or h > img_highres.shape[0] * 0.9:
            continue
        if area < 100:
            continue
        boxes.append((x, y, w, h))

    # 🔧 اگر خیلی کانتور داریم، فقط 8 تا از مناسب‌ترین‌ها رو بگیریم
    if len(boxes) > 8:
        boxes = sorted(boxes, key=lambda b: b[2]*b[3], reverse=True)  # بر اساس مساحت
        boxes = boxes[:8]

    # 🔧 مرتب‌سازی از چپ به راست
    boxes = sorted(boxes, key=lambda b: b[0])

    count = 0
    pad = 6
    for i, (x, y, w, h) in enumerate(boxes):
        x_new = max(x - pad, 0)
        y_new = max(y - pad, 0)
        w_new = min(w + 2 * pad, img_highres.shape[1] - x_new)
        h_new = min(h + 2 * pad, img_highres.shape[0] - y_new)

        char_img = img_highres[y_new:y_new + h_new, x_new:x_new + w_new]

        # تغییر اندازه و شارپ کردن
        char_img_resized = cv2.resize(char_img, (64, 128), interpolation=cv2.INTER_CUBIC)
        kernel_sharp = np.array([[0, -1, 0],
                                 [-1, 5, -1],
                                 [0, -1, 0]])
        char_img_resized = cv2.filter2D(char_img_resized, -1, kernel_sharp)

        out_name = f"{os.path.splitext(filename)[0]}_char{count + 1}.png"
        cv2.imwrite(os.path.join(output_folder, out_name), char_img_resized)
        count += 1

    print(f"✅ {filename}: {count} کاراکتر نهایی ذخیره شد.")

print("🎯 پردازش کامل شد - دقیق و محدود به 8 کاراکتر.")


✅ 1.png: 8 کاراکتر نهایی ذخیره شد.
✅ 10.png: 8 کاراکتر نهایی ذخیره شد.
✅ 100.png: 8 کاراکتر نهایی ذخیره شد.
✅ 101.png: 8 کاراکتر نهایی ذخیره شد.
✅ 102.png: 8 کاراکتر نهایی ذخیره شد.
✅ 103.png: 8 کاراکتر نهایی ذخیره شد.
✅ 104.png: 8 کاراکتر نهایی ذخیره شد.
✅ 105.png: 8 کاراکتر نهایی ذخیره شد.
✅ 106.png: 2 کاراکتر نهایی ذخیره شد.
✅ 107.png: 8 کاراکتر نهایی ذخیره شد.
✅ 108.png: 8 کاراکتر نهایی ذخیره شد.
✅ 109.png: 8 کاراکتر نهایی ذخیره شد.
✅ 11.png: 8 کاراکتر نهایی ذخیره شد.
✅ 110.png: 5 کاراکتر نهایی ذخیره شد.
✅ 111.png: 8 کاراکتر نهایی ذخیره شد.
✅ 112.png: 8 کاراکتر نهایی ذخیره شد.
✅ 113.png: 8 کاراکتر نهایی ذخیره شد.
✅ 114.png: 8 کاراکتر نهایی ذخیره شد.
✅ 115.png: 7 کاراکتر نهایی ذخیره شد.
✅ 116.png: 8 کاراکتر نهایی ذخیره شد.
✅ 117.png: 8 کاراکتر نهایی ذخیره شد.
✅ 118.png: 8 کاراکتر نهایی ذخیره شد.
✅ 119.png: 8 کاراکتر نهایی ذخیره شد.
✅ 12.png: 8 کاراکتر نهایی ذخیره شد.
✅ 120.png: 8 کاراکتر نهایی ذخیره شد.
✅ 121.png: 8 کاراکتر نهایی ذخیره شد.
✅ 122.png: 8 کاراکتر نهایی ذخیره شد.
✅ 123.

### تنظیم سایز تصاویر

In [50]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [53]:

# مسیرها
input_dir = 'output_phase2.3'
output_dir = 'output_phase2.4'
os.makedirs(output_dir, exist_ok=True)

def clean_and_center_image(img, size=(28, 28)):
    # تبدیل به خاکستری اگر رنگی بود
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # باینری کردن (متن سیاه، زمینه سفید)
    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # حذف نویز با Morphology
    kernel = np.ones((2, 2), np.uint8)
    clean = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    # پیدا کردن بزرگ‌ترین کانتور (کاراکتر اصلی)
    contours, _ = cv2.findContours(clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return np.ones(size, dtype=np.uint8) * 255  # برگرد تصویر سفید خالی

    # انتخاب بزرگ‌ترین کانتور
    main_contour = max(contours, key=cv2.contourArea)
    mask = np.zeros_like(clean)
    cv2.drawContours(mask, [main_contour], -1, 255, thickness=cv2.FILLED)

    # فقط کاراکتر اصلی نگه داشته بشه
    isolated = cv2.bitwise_and(clean, mask)

    # کراپ به اطراف کاراکتر
    x, y, w, h = cv2.boundingRect(main_contour)
    char_img = isolated[y:y+h, x:x+w]

    # ریسایز با حفظ نسبت
    h_new, w_new = char_img.shape
    scale = min((size[0] - 4) / h_new, (size[1] - 4) / w_new)
    char_resized = cv2.resize(char_img, (int(w_new * scale), int(h_new * scale)), interpolation=cv2.INTER_AREA)

    # قرار دادن وسط تصویر سفید
    final_img = np.ones(size, dtype=np.uint8) * 255
    h_final, w_final = char_resized.shape
    y_offset = (size[0] - h_final) // 2
    x_offset = (size[1] - w_final) // 2
    final_img[y_offset:y_offset+h_final, x_offset:x_offset+w_final] = 255 - char_resized  # برعکسش کنیم چون اینورت شده بود

    return final_img

# پردازش تصاویر
for filename in tqdm(os.listdir(input_dir)):
    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename)

    img = cv2.imread(input_path)
    if img is None:
        continue

    cleaned_img = clean_and_center_image(img)
    cv2.imwrite(output_path, cleaned_img)


100%|██████████| 1612/1612 [00:01<00:00, 1402.84it/s]


# فاز 3

### آماده سازی داده ها

In [58]:
import os
from PIL import Image

In [59]:
import os
from PIL import Image, ImageChops

input_dir = 'alpha'
output_dir = 'output_phase3'

os.makedirs(output_dir, exist_ok=True)

def crop_white_borders(image):
    """برش دادن نواحی سفید اطراف تصویر"""
    bg = Image.new(image.mode, image.size, (255, 255, 255))  # زمینه سفید
    diff = ImageChops.difference(image, bg)
    bbox = diff.getbbox()
    return image.crop(bbox) if bbox else image

def paste_on_white_canvas(image, size=(28, 28)):
    """قرار دادن تصویر در بوم سفید و وسط‌چین کردن آن"""
    canvas = Image.new("RGB", size, (255, 255, 255))  # ← بوم سفید
    img_w, img_h = image.size
    scale = min(size[0] / img_w, size[1] / img_h)
    new_size = (int(img_w * scale), int(img_h * scale))
    image = image.resize(new_size, Image.LANCZOS)

    paste_x = (size[0] - new_size[0]) // 2
    paste_y = (size[1] - new_size[1]) // 2
    canvas.paste(image, (paste_x, paste_y))
    return canvas

for folder_name in os.listdir(input_dir):
    input_folder_path = os.path.join(input_dir, folder_name)
    
    if os.path.isdir(input_folder_path):
        output_folder_path = os.path.join(output_dir, folder_name)
        os.makedirs(output_folder_path, exist_ok=True)

        for img_name in os.listdir(input_folder_path):
            input_img_path = os.path.join(input_folder_path, img_name)
            output_img_path = os.path.join(output_folder_path, img_name)

            try:
                with Image.open(input_img_path) as img:
                    img = img.convert("RGB")
                    cropped_img = crop_white_borders(img)
                    final_img = paste_on_white_canvas(cropped_img, size=(28, 28))
                    final_img.save(output_img_path)
            except Exception as e:
                print(f"خطا در پردازش عکس {input_img_path}: {e}")


### اموزش مدل

In [61]:
import os
import numpy as np
import cv2
from tqdm import tqdm
from skimage.feature import hog
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib


In [62]:

# مسیرها
input_dir = 'output_phase3'                 # داده‌های آموزشی
output_dir = 'output_phase3.1'              # ذخیره مدل‌ها
os.makedirs(output_dir, exist_ok=True)

# پارامترهای HOG
image_size = (28, 28)
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

X = []
y = []

# بارگذاری برچسب‌ها از نام پوشه‌ها
label_names = sorted(os.listdir(input_dir))
label_map = {name: idx for idx, name in enumerate(label_names)}

def preprocess_image(img):
    _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    x, y_, w, h = cv2.boundingRect(binary)
    cropped = binary[y_:y_+h, x:x+w]
    resized = cv2.resize(cropped, image_size)
    return resized

def augment_image(img):
    rows, cols = img.shape
    angle = np.random.uniform(-10, 10)
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1.0)
    return cv2.warpAffine(img, M, (cols, rows), borderValue=255)

print("🚀 در حال پردازش تصاویر...")

for label_name in tqdm(label_names):
    folder_path = os.path.join(input_dir, label_name)
    if not os.path.isdir(folder_path):
        continue
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        try:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue

            pre_img = preprocess_image(img)

            # ویژگی اصلی
            features = hog(pre_img,
                           orientations=orientations,
                           pixels_per_cell=pixels_per_cell,
                           cells_per_block=cells_per_block,
                           block_norm='L2-Hys',
                           visualize=False)
            X.append(features)
            y.append(label_map[label_name])

            # نسخه چرخش‌دار
            aug_img = augment_image(pre_img)
            features_aug = hog(aug_img,
                               orientations=orientations,
                               pixels_per_cell=pixels_per_cell,
                               cells_per_block=cells_per_block,
                               block_norm='L2-Hys',
                               visualize=False)
            X.append(features_aug)
            y.append(label_map[label_name])

        except Exception as e:
            print(f"⚠️ خطا در پردازش {img_path}: {e}")

X = np.array(X)
y = np.array(y)

print(f"\n📊 تعداد کل نمونه‌ها (با Augmentation): {X.shape[0]}")

# کاهش بعد با PCA
print("⚙️ در حال اعمال PCA...")
pca = PCA(n_components=50)
X_pca = pca.fit_transform(X)

# تقسیم آموزش/تست
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# آموزش SVM
print("\n🎓 آموزش SVM...")
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)

# دقت SVM
svm_train_acc = accuracy_score(y_train, svm_model.predict(X_train))
svm_test_acc = accuracy_score(y_test, svm_model.predict(X_test))
print(f"🎯 دقت SVM روی داده‌های آموزش: {svm_train_acc:.2f}")
print(f"🎯 دقت SVM روی داده‌های تست: {svm_test_acc:.2f}")

# آموزش درخت تصمیم با کنترل Overfitting با pruning و محدودیت‌های بهینه
print("\n🎓 آموزش درخت تصمیم (با جلوگیری از Overfitting)...")
dt_model = DecisionTreeClassifier(
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    ccp_alpha=0.001,  # pruning
    random_state=42
)
dt_model.fit(X_train, y_train)

dt_train_acc = accuracy_score(y_train, dt_model.predict(X_train))
dt_test_acc = accuracy_score(y_test, dt_model.predict(X_test))
print(f"🌳 دقت درخت تصمیم روی داده‌های آموزش: {dt_train_acc:.2f}")
print(f"🌳 دقت درخت تصمیم روی داده‌های تست: {dt_test_acc:.2f}")

# استفاده از GridSearchCV برای تنظیم Random Forest
print("\n🔍 جستجوی بهترین تنظیمات Random Forest با GridSearch...")
param_grid = {
    'n_estimators': [100, 150],
    'max_depth': [10, 20],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 5]
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=3,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
rf_model = grid_search.best_estimator_

# دقت Random Forest
rf_train_acc = accuracy_score(y_train, rf_model.predict(X_train))
rf_test_acc = accuracy_score(y_test, rf_model.predict(X_test))
print(f"🌲 بهترین پارامترهای Random Forest: {grid_search.best_params_}")
print(f"🌲 دقت Random Forest روی داده‌های آموزش: {rf_train_acc:.2f}")
print(f"🌲 دقت Random Forest روی داده‌های تست: {rf_test_acc:.2f}")

# ذخیره مدل‌ها
joblib.dump(svm_model, os.path.join(output_dir, 'svm_model.joblib'))
joblib.dump(dt_model, os.path.join(output_dir, 'decision_tree_model.joblib'))
joblib.dump(rf_model, os.path.join(output_dir, 'random_forest_model.joblib'))
joblib.dump(pca, os.path.join(output_dir, 'pca_transform.joblib'))

print(f"\n✅ مدل‌ها و PCA در مسیر {output_dir} ذخیره شدند.")


🚀 در حال پردازش تصاویر...


  0%|          | 0/39 [00:00<?, ?it/s]

100%|██████████| 39/39 [00:02<00:00, 17.13it/s]



📊 تعداد کل نمونه‌ها (با Augmentation): 7798
⚙️ در حال اعمال PCA...

🎓 آموزش SVM...
🎯 دقت SVM روی داده‌های آموزش: 0.98
🎯 دقت SVM روی داده‌های تست: 0.96

🎓 آموزش درخت تصمیم (با جلوگیری از Overfitting)...
🌳 دقت درخت تصمیم روی داده‌های آموزش: 0.84
🌳 دقت درخت تصمیم روی داده‌های تست: 0.76

🔍 جستجوی بهترین تنظیمات Random Forest با GridSearch...
Fitting 3 folds for each of 16 candidates, totalling 48 fits
🌲 بهترین پارامترهای Random Forest: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 150}
🌲 دقت Random Forest روی داده‌های آموزش: 1.00
🌲 دقت Random Forest روی داده‌های تست: 0.93

✅ مدل‌ها و PCA در مسیر output_phase3.1 ذخیره شدند.


# فاز 4

### نام دادن کاراکتر ها با مدل های خودمون

In [63]:
import os
import cv2
import numpy as np
from skimage.feature import hog
import joblib
from tqdm import tqdm

In [64]:

# مسیرها
input_dir = 'output_phase2.4'
model_dir = 'output_phase3.1'
output_dir = 'output_phase4'
os.makedirs(output_dir, exist_ok=True)

# بارگذاری مدل‌ها و PCA
svm_model = joblib.load(os.path.join(model_dir, 'svm_model.joblib'))
dt_model = joblib.load(os.path.join(model_dir, 'decision_tree_model.joblib'))
rf_model = joblib.load(os.path.join(model_dir, 'random_forest_model.joblib'))  # اضافه شد
pca = joblib.load(os.path.join(model_dir, 'pca_transform.joblib'))

# پارامترهای HOG
image_size = (28, 28)
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

# بارگذاری نام برچسب‌ها
label_names = sorted(os.listdir('output_phase3'))
label_map = {name: idx for idx, name in enumerate(label_names)}
inv_label_map = {v: k for k, v in label_map.items()}

def preprocess_image(img):
    _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    x, y_, w, h = cv2.boundingRect(binary)
    cropped = binary[y_:y_+h, x:x+w]
    resized = cv2.resize(cropped, image_size)
    return resized

# لیست‌های نتایج برای هر مدل
svm_results = []
dt_results = []
rf_results = []  # اضافه شد

print("در حال پردازش تصاویر برای پیش‌بینی برچسب‌ها...")

for img_name in tqdm(os.listdir(input_dir)):
    img_path = os.path.join(input_dir, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"⚠️ نمی‌توان تصویر {img_name} را خواند.")
        continue

    try:
        pre_img = preprocess_image(img)
        features = hog(pre_img,
                       orientations=orientations,
                       pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block,
                       block_norm='L2-Hys',
                       visualize=False)
        features_pca = pca.transform([features])

        svm_pred = svm_model.predict(features_pca)[0]
        dt_pred = dt_model.predict(features_pca)[0]
        rf_pred = rf_model.predict(features_pca)[0]  # اضافه شد

        svm_label = inv_label_map.get(svm_pred, "Unknown")
        dt_label = inv_label_map.get(dt_pred, "Unknown")
        rf_label = inv_label_map.get(rf_pred, "Unknown")  # اضافه شد

        svm_results.append(f"{img_name}\t{svm_label}")
        dt_results.append(f"{img_name}\t{dt_label}")
        rf_results.append(f"{img_name}\t{rf_label}")  # اضافه شد

    except Exception as e:
        print(f"⚠️ خطا در پردازش {img_name}: {e}")

# ذخیره نتایج جداگانه
svm_output_file = os.path.join(output_dir, 'svm_predictions.txt')
dt_output_file = os.path.join(output_dir, 'decision_tree_predictions.txt')
rf_output_file = os.path.join(output_dir, 'random_forest_predictions.txt')  # اضافه شد

with open(svm_output_file, 'w', encoding='utf-8') as f:
    for line in svm_results:
        f.write(line + '\n')

with open(dt_output_file, 'w', encoding='utf-8') as f:
    for line in dt_results:
        f.write(line + '\n')

with open(rf_output_file, 'w', encoding='utf-8') as f:  # اضافه شد
    for line in rf_results:
        f.write(line + '\n')

print(f"\n✅ پیش‌بینی‌های SVM در فایل {svm_output_file} ذخیره شد.")
print(f"✅ پیش‌بینی‌های Decision Tree در فایل {dt_output_file} ذخیره شد.")
print(f"✅ پیش‌بینی‌های Random Forest در فایل {rf_output_file} ذخیره شد.")  # اضافه شد


در حال پردازش تصاویر برای پیش‌بینی برچسب‌ها...


100%|██████████| 1612/1612 [00:08<00:00, 180.79it/s]


✅ پیش‌بینی‌های SVM در فایل output_phase4\svm_predictions.txt ذخیره شد.
✅ پیش‌بینی‌های Decision Tree در فایل output_phase4\decision_tree_predictions.txt ذخیره شد.
✅ پیش‌بینی‌های Random Forest در فایل output_phase4\random_forest_predictions.txt ذخیره شد.


### نام دادن به تصاویر

In [65]:
import os
import shutil


In [66]:

# مسیرها
images_dir = "output_phase2.4"
names_dir = "output_phase4"
output_dir = "output_phase4.1"

# ساختن پوشه خروجی
os.makedirs(output_dir, exist_ok=True)

# خواندن تمام فایل‌های txt در پوشه output_phase4
for txt_file in os.listdir(names_dir):
    if txt_file.endswith(".txt"):
        model_name = os.path.splitext(txt_file)[0]
        model_output_path = os.path.join(output_dir, model_name)
        os.makedirs(model_output_path, exist_ok=True)

        # مسیر فایل txt
        txt_path = os.path.join(names_dir, txt_file)

        # خواندن نام‌ فایل‌ها از txt + پاکسازی
        with open(txt_path, "r", encoding="utf-8") as f:
            lines = [line.strip().replace('\t', '').replace('/', '_').replace('\\', '_') for line in f if line.strip()]

        # پیدا کردن تصاویر به همان ترتیب در پوشه تصاویر
        image_files = sorted(os.listdir(images_dir))  # اگر لازم شد، فیلتر روی فرمت بگذار
        for i, new_name in enumerate(lines):
            if i >= len(image_files):
                print(f"تعداد تصاویر کافی نیست برای {txt_file}")
                break

            original_image_path = os.path.join(images_dir, image_files[i])
            extension = os.path.splitext(image_files[i])[1]
            clean_name = new_name.strip().replace(" ", "_").replace(":", "-")  # بیشتر پاکسازی اگر لازم بود
            new_image_name = f"{clean_name}{extension}"
            new_image_path = os.path.join(model_output_path, new_image_name)

            try:
                shutil.copy(original_image_path, new_image_path)
            except Exception as e:
                print(f"❌ خطا در کپی {original_image_path} به {new_image_path}: {e}")

print("✅ تمام شد. تصاویر دسته‌بندی و نام‌گذاری شدند.")


✅ تمام شد. تصاویر دسته‌بندی و نام‌گذاری شدند.


### نمایش کاراکتر های پلاک به در یک خط

In [12]:
import os
import cv2
import numpy as np
from skimage.feature import hog
import joblib
from tqdm import tqdm
from collections import defaultdict

In [13]:


# مسیرها
input_dir = 'output_phase2.4'     # تصاویر کاراکترها برای تشخیص
model_dir = 'output_phase3.1'     # مدل‌ها و PCA از فاز 3
output_dir = 'output_phase4.2'      # خروجی فاز 4
os.makedirs(output_dir, exist_ok=True)

# بارگذاری مدل‌ها و PCA
svm_model = joblib.load(os.path.join(model_dir, 'svm_model.joblib'))
dt_model = joblib.load(os.path.join(model_dir, 'decision_tree_model.joblib'))
rf_model = joblib.load(os.path.join(model_dir, 'random_forest_model.joblib'))  # اضافه شده
pca = joblib.load(os.path.join(model_dir, 'pca_transform.joblib'))

# پارامترهای HOG
image_size = (28, 28)
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

# بارگذاری نام برچسب‌ها
label_names = sorted(os.listdir('output_phase3'))
label_map = {name: idx for idx, name in enumerate(label_names)}
inv_label_map = {v: k for k, v in label_map.items()}

def preprocess_image(img):
    _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    x, y_, w, h = cv2.boundingRect(binary)
    cropped = binary[y_:y_+h, x:x+w]
    resized = cv2.resize(cropped, image_size)
    return resized

# برای ذخیره پیش‌بینی‌ها
svm_results = defaultdict(list)
dt_results = defaultdict(list)
rf_results = defaultdict(list)  # اضافه شده

print("در حال پردازش تصاویر برای پیش‌بینی برچسب‌ها...")

# مرتب‌سازی فایل‌ها برای ترتیب صحیح کاراکترها
image_files = sorted(os.listdir(input_dir))

for img_name in tqdm(image_files):
    img_path = os.path.join(input_dir, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"⚠️ نمی‌توان تصویر {img_name} را خواند.")
        continue

    try:
        plate_id = img_name.split('_')[0]  # مثل 101 از "101_char5.png"
        pre_img = preprocess_image(img)
        features = hog(pre_img,
                       orientations=orientations,
                       pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block,
                       block_norm='L2-Hys',
                       visualize=False)
        features_pca = pca.transform([features])

        svm_pred = svm_model.predict(features_pca)[0]
        dt_pred = dt_model.predict(features_pca)[0]
        rf_pred = rf_model.predict(features_pca)[0]  # اضافه شده

        svm_label = inv_label_map.get(svm_pred, "Unknown")
        dt_label = inv_label_map.get(dt_pred, "Unknown")
        rf_label = inv_label_map.get(rf_pred, "Unknown")  # اضافه شده

        svm_results[plate_id].append(svm_label)
        dt_results[plate_id].append(dt_label)
        rf_results[plate_id].append(rf_label)  # اضافه شده

    except Exception as e:
        print(f"⚠️ خطا در پردازش {img_name}: {e}")

# نوشتن خروجی‌ها
svm_output_path = os.path.join(output_dir, 'svm_predictions.txt')
dt_output_path = os.path.join(output_dir, 'dt_predictions.txt')
rf_output_path = os.path.join(output_dir, 'random_forest_predictions.txt')  # اضافه شده

with open(svm_output_path, 'w', encoding='utf-8') as f_svm, \
     open(dt_output_path, 'w', encoding='utf-8') as f_dt, \
     open(rf_output_path, 'w', encoding='utf-8') as f_rf:  # اضافه شده
    f_svm.write("name,plate\n")
    f_dt.write("name,plate\n")
    f_rf.write("name,plate\n")  # اضافه شده

    for plate_id in sorted(svm_results.keys(), key=lambda x: int(x)):
        svm_line = ','.join([plate_id] + svm_results[plate_id])
        dt_line = ','.join([plate_id] + dt_results[plate_id])
        rf_line = ','.join([plate_id] + rf_results[plate_id])  # اضافه شده
        f_svm.write(svm_line + '\n')
        f_dt.write(dt_line + '\n')
        f_rf.write(rf_line + '\n')  # اضافه شده

print("\n✅ فایل‌های زیر ساخته شدند:")
print("📄", svm_output_path)
print("📄", dt_output_path)
print("📄", rf_output_path)  # اضافه شده


در حال پردازش تصاویر برای پیش‌بینی برچسب‌ها...


100%|██████████| 1612/1612 [00:06<00:00, 264.19it/s]


✅ فایل‌های زیر ساخته شدند:
📄 output_phase4.2\svm_predictions.txt
📄 output_phase4.2\dt_predictions.txt
📄 output_phase4.2\random_forest_predictions.txt


### تصحیح نام کاراکتر ها

In [15]:
import os


In [17]:

input_dir = "output_phase4.2"
output_dir = "output_phase4.3"
os.makedirs(output_dir, exist_ok=True)

model_files = {
    "svm": "svm_predictions.txt",
    "random_forest": "random_forest_predictions.txt",
    "dt": "dt_predictions.txt"
}

def clean_labels_from_file(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as fin, open(output_path, "w", encoding="utf-8") as fout:
        header = fin.readline()
        fout.write(header)  # نوشتن header به فایل خروجی

        for line in fin:
            parts = line.strip().split(",")
            name = parts[0]
            labels = parts[1:]
            cleaned_labels = []
            for label in labels:
                if '-' in label:
                    cleaned_labels.append(label.split('-', 1)[1])
                else:
                    cleaned_labels.append(label)
            fout.write(name + "," + ",".join(cleaned_labels) + "\n")

for model_name, filename in model_files.items():
    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename)
    if os.path.exists(input_path):
        clean_labels_from_file(input_path, output_path)
        print(f"✅ فایل {filename} پردازش و ذخیره شد در {output_path}")
    else:
        print(f"⚠️ فایل {filename} در مسیر {input_dir} یافت نشد!")


✅ فایل svm_predictions.txt پردازش و ذخیره شد در output_phase4.3\svm_predictions.txt
✅ فایل random_forest_predictions.txt پردازش و ذخیره شد در output_phase4.3\random_forest_predictions.txt
✅ فایل dt_predictions.txt پردازش و ذخیره شد در output_phase4.3\dt_predictions.txt


# فاز 5

In [18]:
import os
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [19]:
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# مسیر فایل‌ها
svm_pred_path = "output_phase4.3/svm_predictions.txt"
dt_pred_path = "output_phase4.3/dt_predictions.txt"
rf_pred_path = "output_phase4.3/random_forest_predictions.txt"
label_path = "Plates2/plate_labels.txt"
output_dir = "output_phase5"

# ایجاد پوشه خروجی در صورت عدم وجود
os.makedirs(output_dir, exist_ok=True)

# تابع خواندن فایل‌ها
def read_labels(filepath):
    with open(filepath, "r") as f:
        return [line.strip() for line in f.readlines()]

# خواندن داده‌ها
y_true = read_labels(label_path)
y_pred_svm = read_labels(svm_pred_path)
y_pred_dt = read_labels(dt_pred_path)
y_pred_rf = read_labels(rf_pred_path)

# فیلتر پلاک‌هایی که طول کاراکترها برابر است
def filter_matching_length(true_labels, pred_labels):
    filtered_true = []
    filtered_pred = []
    for t, p in zip(true_labels, pred_labels):
        if len(t) == len(p):
            filtered_true.append(t)
            filtered_pred.append(p)
    return filtered_true, filtered_pred

# تبدیل لیست به کاراکترهای جداگانه
def flatten(chars_list):
    return [char for plate in chars_list for char in plate]

# ارزیابی مدل
def evaluate_model(y_true_chars, y_pred_chars, model_name):
    acc = accuracy_score(y_true_chars, y_pred_chars)
    prec = precision_score(y_true_chars, y_pred_chars, average='macro', zero_division=0)
    rec = recall_score(y_true_chars, y_pred_chars, average='macro', zero_division=0)
    cm = confusion_matrix(y_true_chars, y_pred_chars)

    # ذخیره فایل متریک‌ها
    with open(f"{output_dir}/{model_name}_metrics.txt", "w") as f:
        f.write(f"Accuracy: {acc:.4f}\n")
        f.write(f"Precision: {prec:.4f}\n")
        f.write(f"Recall: {rec:.4f}\n")

    # رسم confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
    plt.title(f"Confusion Matrix - {model_name.upper()}")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{model_name}_confusion_matrix.png")
    plt.close()

    # چاپ درصدها در ترمینال
    print(f"\n🔍 {model_name.upper()} Evaluation:")
    print(f"  ✅ Accuracy : {acc * 100:.2f}%")
    print(f"  🎯 Precision: {prec * 100:.2f}%")
    print(f"  🔁 Recall   : {rec * 100:.2f}%")

# ===========================
# ارزیابی SVM
y_true_svm, y_pred_svm_filtered = filter_matching_length(y_true, y_pred_svm)
print(f"SVM: {len(y_true_svm)} samples after filtering.")
y_true_chars_svm = flatten(y_true_svm)
y_pred_svm_chars = flatten(y_pred_svm_filtered)

if y_true_chars_svm and y_pred_svm_chars:
    evaluate_model(y_true_chars_svm, y_pred_svm_chars, "svm")
else:
    print("⚠️ SVM: No valid data after filtering for matching lengths.")

# ارزیابی Decision Tree
y_true_dt, y_pred_dt_filtered = filter_matching_length(y_true, y_pred_dt)
print(f"Decision Tree: {len(y_true_dt)} samples after filtering.")
y_true_chars_dt = flatten(y_true_dt)
y_pred_dt_chars = flatten(y_pred_dt_filtered)

if y_true_chars_dt and y_pred_dt_chars:
    evaluate_model(y_true_chars_dt, y_pred_dt_chars, "dt")
else:
    print("⚠️ Decision Tree: No valid data after filtering for matching lengths.")

# ارزیابی Random Forest
y_true_rf, y_pred_rf_filtered = filter_matching_length(y_true, y_pred_rf)
print(f"Random Forest: {len(y_true_rf)} samples after filtering.")
y_true_chars_rf = flatten(y_true_rf)
y_pred_rf_chars = flatten(y_pred_rf_filtered)

if y_true_chars_rf and y_pred_rf_chars:
    evaluate_model(y_true_chars_rf, y_pred_rf_chars, "random_forest")
else:
    print("⚠️ Random Forest: No valid data after filtering for matching lengths.")


SVM: 8 samples after filtering.

🔍 SVM Evaluation:
  ✅ Accuracy : 23.17%
  🎯 Precision: 7.65%
  🔁 Recall   : 6.97%
Decision Tree: 6 samples after filtering.

🔍 DT Evaluation:
  ✅ Accuracy : 12.66%
  🎯 Precision: 8.69%
  🔁 Recall   : 7.44%
Random Forest: 9 samples after filtering.

🔍 RANDOM_FOREST Evaluation:
  ✅ Accuracy : 7.78%
  🎯 Precision: 1.79%
  🔁 Recall   : 1.39%
